In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
from pyspark.sql.functions import col, avg, count

spark= SparkSession \
       .builder \
       .appName("BestAndWorstProducts") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,400 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-secur

In [ ]:



# Load the JSON file
df = spark.read.json('reviews_MP3_Players_and_Accessories.json')

df.show()


+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin|helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+-------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|9861006532| [0, 0]|    4.0|Love the way it l...|03 16, 2010| AEIPMJVXOH3Y5|Amanda J. Rapp "A...|         Cherry Case|    1268697600|
|9861029672| [0, 0]|    5.0|this is a great c...|02 10, 2012|A1Z4KYGN1GLVAK|           Doreend06|       great product|    1328832000|
|9882076335| [0, 0]|    4.0|I ordered this ca...|02 12, 2011|  A6M6F6RMKN77|             Katie18|         Looks real!|    1297468800|
|9882076335| [0, 0]|    1.0|Thsi iphone case ...| 04 1, 2011|A1X3SJZWSM8FJZ|                QUAZ|   It's HORRIBLE!!!!|    1301616000|
|9882076335| [0, 0]|    1.0|first, it looked ...|12 27, 2010|A

In [ ]:
grouped_df = df.groupBy("asin").agg(
    count("*").alias("num_reviews"),
    avg("overall").alias("avg_rating")
)

# Show the aggregated DataFrame
grouped_df.show(truncate=False)

+----------+-----------+------------------+
|asin      |num_reviews|avg_rating        |
+----------+-----------+------------------+
|B0018MESWU|1          |5.0               |
|B001DJDTZK|1          |1.0               |
|B001EHLSQS|1          |5.0               |
|B002P0ZK4Y|2          |3.5               |
|B0037FY09M|3          |4.0               |
|B004IM7X7E|13         |3.0               |
|B00006I512|1          |5.0               |
|B0006FLOR2|6          |3.5               |
|B000A1ICRC|1          |2.0               |
|B000BYU63Q|22         |3.909090909090909 |
|B0033XWK40|1          |3.0               |
|B005G4XEV2|7          |4.571428571428571 |
|B00005AW1E|55         |2.3636363636363638|
|B0000C9Z2Z|1          |5.0               |
|B0001NA3EE|2          |5.0               |
|B000B45CKI|1          |5.0               |
|B000NNRCF0|1          |2.0               |
|B004BOG0Z0|1          |5.0               |
|B004WP2D6I|3          |1.3333333333333333|
|B00004WHW2|8          |3.875   

In [ ]:
# Get top 10 and worst 10 by number of reviews
top_10_reviews = grouped_df.orderBy(col("num_reviews").desc()).limit(10)
worst_10_reviews = grouped_df.orderBy(col("num_reviews").asc()).limit(10)

# Save the results to CSV files
top_10_reviews.coalesce(1).write.csv("top_10_reviews.csv", header=True)
worst_10_reviews.coalesce(1).write.csv("worst_10_reviews.csv", header=True)


In [ ]:
# Get top 10 and worst 10 by average rating
top_10_ratings = grouped_df.orderBy(col("avg_rating").desc()).limit(10)
worst_10_ratings = grouped_df.orderBy(col("avg_rating").asc()).limit(10)
#save
top_10_ratings.coalesce(1).write.csv("top_10_ratings.csv", header=True)
worst_10_ratings.coalesce(1).write.csv("worst_10_ratings.csv", header=True)

In [ ]:
top_10_ratings.show()
worst_10_ratings.show()

+----------+-----------+----------+
|      asin|num_reviews|avg_rating|
+----------+-----------+----------+
|B000VE0G4K|          1|       5.0|
|B0050P5Z3W|          1|       5.0|
|B0018MESWU|          1|       5.0|
|B000CS9K8I|          1|       5.0|
|B001EHLSQS|          1|       5.0|
|B004BOG0Z0|          1|       5.0|
|B0002NRL9S|          1|       5.0|
|B0000C9Z2Z|          1|       5.0|
|B000NP2JSI|          2|       5.0|
|B000B45CKI|          1|       5.0|
+----------+-----------+----------+

+----------+-----------+----------+
|      asin|num_reviews|avg_rating|
+----------+-----------+----------+
|B00112AZ7Y|          2|       1.0|
|B00346GQUK|          1|       1.0|
|B0000T70QI|          2|       1.0|
|B002AT26AG|          1|       1.0|
|B001DJDTZK|          1|       1.0|
|B002PO5B6M|          1|       1.0|
|B0049IG6V6|          1|       1.0|
|B004OYPDZ0|          1|       1.0|
|B000G3KNEY|          1|       1.0|
|B0044Y5CUG|          1|       1.0|
+----------+-----------+---

In [ ]:
top_10_reviews.show()
worst_10_reviews.show()

+----------+-----------+------------------+
|      asin|num_reviews|        avg_rating|
+----------+-----------+------------------+
|B007R6HUBC|       3147| 4.236097870988242|
|B0042B9U8Q|       2263| 3.152894387980557|
|B003N9SR00|       1191| 4.216624685138539|
|B005P1VNDW|        564| 4.340425531914893|
|B0097BEG08|        545|3.9706422018348624|
|B001DDE4VY|        434| 4.071428571428571|
|B008XMV05G|        409| 4.124694376528117|
|B00277EMQ2|        369|3.6151761517615175|
|B005NGJLYW|        306| 3.349673202614379|
|B005P1VMLU|        237|4.4978902953586495|
+----------+-----------+------------------+

+----------+-----------+----------+
|      asin|num_reviews|avg_rating|
+----------+-----------+----------+
|B0033XWK40|          1|       3.0|
|B000VE0G4K|          1|       5.0|
|B001DJDTZK|          1|       1.0|
|B0000C9Z2Z|          1|       5.0|
|B004BOG0Z0|          1|       5.0|
|B00006I512|          1|       5.0|
|B000NNRCF0|          1|       2.0|
|B000B45CKI|          1